In [0]:
from pyspark.sql import functions as F

In [0]:
base = spark.read.table("stream1.stream1.base_for_gold")
base.printSchema()

In [0]:
customer_summary = (
    base.groupBy("customer_id", "full_name", "email", "currency")
    .agg(
        F.sum("wallet_balance").alias("total_wallet_balance"),
        F.countDistinct("trade_id").alias("total_trades"),
        F.round(F.avg("trade_fee"), 4).alias("avg_trade_fee"),
        F.sum("trade_value").alias("total_trade_value")
    )
)
customer_summary.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("stream1.stream1.gold_customer_summary")
display(customer_summary)

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
asset_summary = (
    base.groupBy("asset_name")
    .agg(
        F.countDistinct("trade_id").alias("trade_count"),
        F.round(F.avg("price_value"), 2).alias("avg_price"),
        F.round(F.sum("trade_value"), 2).alias("total_trade_value"),
        F.round(F.sum("order_quantity"), 4).alias("total_quantity")
    )
)
asset_summary.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("stream1.stream1.gold_asset_summary")
display(asset_summary)

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
order_summary = (
    base.groupBy("order_type", "order_status","asset_name")
    .agg(
        F.countDistinct("order_id").alias("total_orders"),
        
        F.sum("trade_value").alias("total_trade_value"),
        F.avg("trade_fee").alias("avg_fee")
    )
)
order_summary.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("stream1.stream1.gold_order_summary")
display(order_summary)

Databricks visualization. Run in Databricks to view.

In [0]:
# Calculate daily trade trend per asset
daily_trade_trend = (
    base
    # Convert trade_date to date type
    .withColumn("trade_date", F.to_date("trade_date"))
    # Group by trade_date and asset_name
    .groupBy("trade_date", "asset_name")
    # Aggregate daily trade value and average daily price
    .agg(
        F.sum("trade_value").alias("daily_trade_value"),
        F.avg("price_value").alias("avg_daily_price")
    )
)
# Write the result to a Delta table (uncomment to enable)
daily_trade_trend.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("stream1.stream1.gold_daily_trade_trend")
display(daily_trade_trend)
